In [9]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [10]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 29/12/22 - Horário: 11:38:16.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:06.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:10.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:20.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:42:20.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:35.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:48.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:44:27.  Portanto não foi baixado novamente.
vrapeelo

In [11]:
ide_data_cols = ['DthEnvio','IdeUsinaOutorga']

In [12]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)

In [13]:
cols = ['IdeUsinaOutorga','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DthEnvio' : 'Data_Envio',
        'NomUsina' : 'usina_nome',
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)

In [14]:
cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)


cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

input_rapeel = pd.merge(mercado,ambiental_datas,on=ide_data_cols,how="inner")
input_rapeel = pd.merge(input_rapeel,acesso_contratos,on=ide_data_cols,how="inner")
input_rapeel = input_rapeel.loc[input_rapeel.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]


In [15]:
cols = ["IdeUsinaOutorga","DthEnvio","DatPrevistaComercial"]
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]

In [16]:
definicoes = pd.merge(input_rapeel,ug_rapeel,on=ide_data_cols,how='inner')
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]

In [17]:
cols = ["DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","DatMonitoramento","IdeUsinaOutorga","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado"]

monitoramentodados = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]

In [18]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao
rename_cols = {'DatPrevistaComercial':'DatPrevistaComercialRapeelMX'}
definicoes_monitoramento = pd.merge(monitoramentodados,definicoes,on='IdeUsinaOutorga',how="left").rename(columns=rename_cols)
definicoes_monitoramento = definicoes_monitoramento.loc[definicoes_monitoramento.IdcUsinaMonitorada == "Sim"].copy()

definicoes_monitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoes_monitoramento.loc[definicoes_monitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]
definicoes_monitoramento.loc[definicoes_monitoramento.CondicaoAmbiental.isna(),'CondicaoAmbiental'] = 'Sem LP'
definicoes_monitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)


definicoes_monitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSitContratoCCD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCCT == nsa) &
    definicoes_monitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoes_monitoramento.loc[
(    ((definicoes_monitoramento.NomSitContratoCCD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCCT == valido)) &
    definicoes_monitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSitContratoCCD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCCT == vencido) |
    (definicoes_monitoramento.NomSitContratoCCD == nass) |
    (definicoes_monitoramento.NomSitContratoCCT == nass )) &
    definicoes_monitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoes_monitoramento[['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoes_monitoramento['CondicaoUso'] = "Não informado"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCUST == vencido) |
    (definicoes_monitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoes_monitoramento.NomSitContratoCUST == nass )) &
    definicoes_monitoramento.DthEnvio.notna())
    ,'CondicaoUso'] = "Sem Uso"

display(definicoes_monitoramento[['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoes_monitoramento["PPA"] = "Ambos"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoes_monitoramento[['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
48251,Não se Aplica,Não Assinado,2022-12-02 13:46:07.173,Sem Conexão
21505,Não se Aplica,Válido,2022-12-05 17:06:30.997,OK
43351,Não se Aplica,Válido,2022-12-05 16:17:22.260,OK
74634,Não se Aplica,Não Assinado,2022-12-02 23:25:52.027,Sem Conexão
15622,Não se Aplica,Válido,2022-12-01 16:08:06.183,OK
32626,Não se Aplica,Válido,2022-12-04 18:07:10.277,OK
12115,Não se Aplica,Não Assinado,2022-12-04 21:48:50.997,Sem Conexão
44161,Não se Aplica,Válido,2022-12-05 14:00:39.787,OK
590,Válido,Não se Aplica,2022-12-04 19:53:25.833,OK
75701,Não se Aplica,Não Assinado,2022-12-02 23:47:57.877,Sem Conexão


,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
44633,Não se Aplica,Válido,Sem Conexão
1129,Não se Aplica,Válido,OK
54937,Não se Aplica,Não Assinado,Sem Conexão
81598,Não se Aplica,Não Assinado,Sem Conexão
54827,Não se Aplica,Não Assinado,Sem Conexão
10601,Não se Aplica,Não Assinado,Sem Conexão
59530,Não se Aplica,Não Assinado,Sem Conexão
22693,Não se Aplica,Válido,OK
14736,Não se Aplica,Válido,OK
71624,Não se Aplica,Não Assinado,Sem Conexão


,DscComercializacaoEnergia,DatConclusaoACL,PPA
50256,Fora do ACR,NaT,Nenhum
41650,Fora do ACR,NaT,Nenhum
77174,Fora do ACR,NaT,Nenhum
21624,Fora do ACR,2022-07-05,ACL
32713,Fora do ACR,2022-03-18,ACL
62976,Fora do ACR,NaT,Nenhum
38334,Fora do ACR,2021-09-14,ACL
13906,Fora do ACR,2021-05-03,ACL
70438,ACR,NaT,ACR
46764,Fora do ACR,NaT,Nenhum


In [19]:
definicoes_monitoramento[definicoes_monitoramento.IdeUsinaOutorga == 273].drop_duplicates()[['IdeUsinaOutorga','DthEnvio']]

,IdeUsinaOutorga,DthEnvio
0,273,2022-10-09 21:04:17.743
1,273,2022-10-09 21:04:17.743


In [20]:
# Apenas mudança de nome
tabela_criterio = definicoes_monitoramento

In [21]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabela_criterio['criterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabela_criterio['dscriterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabela_criterio[['IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
57550,Em andamento,ACR,LI,Sem Uso,0,Usina em Obras em andamento
62066,Não Iniciada,Nenhum,LP,OK,4,"Sem obras, sem PPA e LP OK e Cust Ok"
12728,Em andamento,ACR,LP,Verificar,0,Usina em Obras em andamento
63512,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
14899,Em andamento,ACL,LO,OK,0,Usina em Obras em andamento
68062,Não Iniciada,ACL,LP,Sem Uso,1,Sem obras e PPA Ok
5028,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
69391,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
9283,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
64357,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"


In [22]:
slp = "Sem LP"
sim = "Sim"
tabela_criterio['criterio_novo'] = np.select(
    [
        (tabela_criterio.IdcSemPrevisao == sim),
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        (tabela_criterio.CondicaoAmbiental == "Sem LP"),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabela_criterio[['IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(10))

,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
60424,Não Iniciada,ACR,LO,Sem Uso,1,Sem obras e PPA Ok
10153,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
66677,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
62262,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
65064,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
59409,Não Iniciada,ACL,LP,Sem Uso,1,Sem obras e PPA Ok
52174,Não Iniciada,Nenhum,Sem LP,Sem Uso,9,Sem LP
15050,Em andamento,ACL,LO,OK,0,Usina em Obras em andamento
67091,Não Iniciada,ACL,LP,Sem Uso,1,Sem obras e PPA Ok
72933,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"


In [39]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)

In [48]:
calculo_previsao[calculo_previsao.Indicador>50]

,NomUsina,IdeUsinaOutorga,NumUgUsina,SigTipoGeracao,Previsao_OC,Dat_OC_obrigacao,DatMonitoramento,FaseAtual,Indicador
0,Emas Nova,27165,9,PCH,2025-03-21,2025-11-18,2022-12-14,OUT_OC,85.977060
1,Bazan,27835,1,UTE,2025-01-24,2025-01-01,2022-12-07,OUT_OC,71.079904
3,Lucia Cherobim,28419,1,PCH,2024-09-22,2024-01-01,2022-12-07,IO_OC,71.732267
7,Córrego Fundo,29505,1,PCH,2025-04-25,2026-01-01,2022-12-13,OUT_OC,86.177175
10,Laguna,30482,3,UTE,2024-03-06,2025-01-01,2022-12-06,IO_OC,85.222173
...,...,...,...,...,...,...,...,...,...
69362,Sol de Itaueira 24,70560,8,UFV,2026-11-18,2027-04-08,2022-12-19,OUT_OC,71.427052
69364,Sol de Itaueira 10,70546,6,UFV,2027-11-13,2027-12-15,2022-12-16,OUT_OC,56.436209
69370,Sol de Itaueira 14,70550,4,UFV,2026-11-18,2027-04-08,2022-12-19,OUT_OC,71.427052
69382,Sol de Itaueira 24,70560,3,UFV,2026-11-18,2027-04-08,2022-12-19,OUT_OC,71.427052


In [ ]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga","DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada"]
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols]

In [ ]:
# vInputCronograma tbm implementado,DATASREAL (LABEL="Idempreendimento" AS Idempreendimento)
cols = ["DatRealizacaoIII","DatRealizacaoV","DatRealizacaoXIV","DatRealizacaoXII","IdeUsinaOutorga","DatRealizacaoIV","DatRealizacaoIX","DatRealizacaoXI","DatRealizacaoX","DthEnvio","DatRealizacaoVI","DatRealizacaoII","CodCeg","DatRealizacaoXIII","DatRealizacaoVII"]
rename_cols = {'DatRealizacaoIII': 'IO_real','DatRealizacaoXIV': 'Conc_Trans_REAL','DatRealizacaoXII': 'Comiss_Real','DatRealizacaoIX': 'DesvRio_real','DatRealizacaoXI': 'Ench_Real','DthEnvio': 'Data_Envio','DatRealizacaoII': 'canteiroReal','DatRealizacaoXIII': 'IOTrans_Real','DatRealizacaoVII': 'ME_Real_conc_eol'}
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates()[cols].rename(columns=rename_cols)
vcronograma["classe"] = vcronograma.CodCeg.str.slice(3,5)
vcronograma["CC_real"] = pd.to_datetime(pd.NA)
vcronograma.loc[vcronograma.classe == "PH","CC_real"] = vcronograma.DatRealizacaoIV
vcronograma.loc[vcronograma.classe == "CV","CC_real"] = vcronograma.DatRealizacaoV
vcronograma["ME_real"] = vcronograma.DatRealizacaoX
vcronograma.loc[vcronograma.classe == "CV","ME_real"] = vcronograma.DatRealizacaoVI

In [ ]:
cols = ["Data_Envio","Comiss_Real","canteiroReal","prev_IO","ME_Real_conc_eol","Conc_Trans_REAL","CC_real","DesvRio_real","Ench_Real","IdeUsinaOutorga","ME_real","IO_real","IOTrans_Real"]
realcronogramarapeel[cols].rename(columns=rename_cols)

NameError: name 'realcronogramarapeel' is not defined

In [ ]:
#WORK.UG_Real n serve pra nd, inputug tbm n 
real_cronograma_rapeel = pd.merge(vcronograma,monitoramentoleilao,on=['IdeUsinaOutorga','Data_Envio'])

KeyError: 'Data_Envio'